여러 배낭 문제로 분배 문제를 해결하려고 노력하였음  
배낭 문제도 동일하게 근본적인 해결 방식 자체가 문제가 있음  

가치와 무게가 서로 다른 여러 아이템들을 어떻게 해야 최대한 가치를 높게 하여 많이 챙길 수 있는가.  
나의 목표를 각 가방마다 동일한 가치를 가지는 것이 문제.  
여러 배낭 문제에서는 서로의 합이 최대가 되도록 문제 해결.  
근본적인 차이가 존재함  
  
타겟 코스트를 동일하게 맞추고 그 이후 위치를 옮기는 방식은 여기서 별로 좋게 작용하지 않음.  
코스트를 완전 내가 임의로 분배한다면 그건 분명한 문제가 생길 터인데.  
일단은 좀 해봐야 알 것 같은걸


In [66]:
import torch
from torch import multiprocessing
import numpy as np
import matplotlib.pyplot as plt


import sys
sys.path.append('/home/ksh-server/workspace/ICUFN')
from my_clustering.my_vmas.scenarios.navigation_random import Scenario

In [67]:
is_fork = multiprocessing.get_start_method() == "fork"
device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)
vmas_device = device

env = Scenario()
world = env.make_world(
        device=device,
        n_agents=4,
        batch_dim=1
    )


In [68]:
def world_reset(env):
    env.reset_world_at()
    agents = env.world.agents
    targets = env.targets
    finished_position = env.finished_pos
    
    return agents, targets, finished_position

In [69]:
def initial_render(agents, targets, finished_position):
    map_size = 2

    # 플롯 생성
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.set_xlim(-map_size, map_size)
    ax.set_ylim(-map_size, map_size)
    ax.set_title("Agent and Target Distribution")

    # 에이전트 플로팅 (파란색 원)
    for i, agent in enumerate(agents):
        x, y = agent.state.pos.squeeze().tolist()
        ax.add_patch(plt.Circle((x, y), agent.shape.radius, color=agent.color, alpha=1, label=f"Agent_{i+1}"))

    for target in targets:
        x, y = target.state.pos.squeeze().tolist()
        label = "target" if targets[0] == target else None
        ax.add_patch(plt.Circle((x, y), target.shape.radius, color="black", alpha=1, label=label))
        
    for pos in finished_position:
        x, y = pos[0].squeeze().tolist()
        label = 'finished_position' if finished_position[0] == pos else None
        ax.add_patch(plt.Circle((x, y), targets[0].shape.radius, color='gray', alpha=1, label=label))

    # 범례 및 표시
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.show()

In [70]:
def numpy_pos(agents, targets, finished_position):
    agents_pos = []
    for agent in agents:
        x, y = agent.state.pos.squeeze().tolist()
        agents_pos.append([x, y])
    agents_pos = np.array(agents_pos)
    
    targets_pos = []
    targets_cost = []
    for target in targets:
        x, y = target.state.pos.squeeze().tolist()
        targets_pos.append([x, y])
        cost = target.cost.item()
        targets_cost.append(cost)
    targets_pos = np.array(targets_pos)
    targets_cost = np.array(targets_cost)

    finished_pos = []
    for pos in finished_position:
        x, y = pos[0].squeeze().tolist()
        finished_pos.append([x, y])
    finished_pos = np.array(finished_pos)
    
    return agents_pos, targets_pos, targets_cost, finished_pos

In [71]:
agents, targets, finished_position = world_reset(env)

In [72]:
agents_pos, targets_pos, targets_cost, finished_pos = numpy_pos(agents, targets, finished_position)


In [73]:
from ortools.linear_solver import pywraplp

In [74]:
targets_cost = targets_cost.tolist()
print(targets_cost)

[10, 10, 10, 20, 20, 20, 30, 30, 30, 40, 40, 40]


In [75]:
targets_values = targets_cost

data = {}
data["weights"] = [10 for _ in range(len(targets_cost))]
data["values"] = targets_cost
assert len(data["weights"]) == len(data["values"])
data["num_items"] = len(data["weights"])
data["all_items"] = range(data["num_items"])

# 수용능력을 어떻게 정해줄 것인가.
# 일단은 100으로 그냥 진행. 공평하게 담아지는가?
data["bin_capacities"] = [30, 30, 30, 30]
data["num_bins"] = len(data["bin_capacities"])
data["all_bins"] = range(data["num_bins"])

In [76]:
solver = pywraplp.Solver.CreateSolver("SCIP")
if solver is None:
    print("SCIP solver unavailable.")
    print("종료시켜라 그냥")
    
x = {}
for i in data["all_items"]:
    for b in data["all_bins"]:
        x[i, b] = solver.BoolVar(f"x_{i}_{b}")

In [77]:
# Constraints.
# Each item is assigned to at most one bin.
for i in data["all_items"]:
    solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

# The amount packed in each bin cannot exceed its capacity.
for b in data["all_bins"]:
    solver.Add(
        sum(x[i, b] * data["weights"][i] for i in data["all_items"])
        <= data["bin_capacities"][b]
    )

# Objective.
# Maximize total value of packed items.
objective = solver.Objective()
for i in data["all_items"]:
    for b in data["all_bins"]:
        objective.SetCoefficient(x[i, b], data["values"][i])
objective.SetMaximization()

print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print(f"Total packed value: {objective.Value()}")
    total_weight = 0
    for b in data["all_bins"]:
        print(f"Bin {b}")
        bin_weight = 0
        bin_value = 0
        for i in data["all_items"]:
            if x[i, b].solution_value() > 0:
                print(
                    f"Item {i} weight: {data['weights'][i]} value:"
                    f" {data['values'][i]}"
                )
                bin_weight += data["weights"][i]
                bin_value += data["values"][i]
        print(f"Packed bin weight: {bin_weight}")
        print(f"Packed bin value: {bin_value}\n")
        total_weight += bin_weight
    print(f"Total packed weight: {total_weight}")
else:
    print("The problem does not have an optimal solution.")

Solving with SCIP 9.2.0 [LP solver: Glop 9.12]
Total packed value: 300.0
Bin 0
Item 0 weight: 10 value: 10
Item 1 weight: 10 value: 10
Item 2 weight: 10 value: 10
Packed bin weight: 30
Packed bin value: 30

Bin 1
Item 3 weight: 10 value: 20
Item 4 weight: 10 value: 20
Item 5 weight: 10 value: 20
Packed bin weight: 30
Packed bin value: 60

Bin 2
Item 6 weight: 10 value: 30
Item 7 weight: 10 value: 30
Item 8 weight: 10 value: 30
Packed bin weight: 30
Packed bin value: 90

Bin 3
Item 9 weight: 10 value: 40
Item 10 weight: 10 value: 40
Item 11 weight: 10 value: 40
Packed bin weight: 30
Packed bin value: 120

Total packed weight: 120
